### Pivot OHLCV Data for Time-Series Analysis

This notebook transforms the cleaned, multi-indexed OHLCV data from a "long" format into a "wide" format. This new format has dates as the index and tickers as columns, which is required by many backtesting and time-series analysis libraries.

**Workflow:**

1.  **Load Data:** Reads the cleaned, multi-indexed OHLCV data.
2.  **Transform:** Selects a specific column (e.g., `Adj Close`) and pivots the DataFrame.
3.  **Save:** Saves the new wide-format DataFrame to a Parquet file.
4.  **Verify:** Reads the saved file back to confirm the process was successful.

### Setup and Configuration

**This is the only cell you need to modify.** Specify the source/destination files and the data column you wish to pivot.

In [1]:
import sys
from pathlib import Path
import pandas as pd

# --- Project Path Setup ---
# Autodetect the project's root directory.
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
DATA_DIR = ROOT_DIR / 'data'
SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# --- File and Column Configuration ---
SOURCE_FILENAME = 'df_OHLCV_clean_stocks_etfs.parquet'
DEST_FILENAME = 'df_adj_close.parquet'
COLUMN_TO_PIVOT = 'Adj Close'  # The data column to extract and pivot

# --- Construct Full Paths ---
SOURCE_PATH = DATA_DIR / SOURCE_FILENAME
DEST_PATH = DATA_DIR / DEST_FILENAME

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Project Root Directory: {ROOT_DIR}")
print(f"Source file: {SOURCE_PATH}")
print(f"Pivoting column: '{COLUMN_TO_PIVOT}'")
print(f"Destination file: {DEST_PATH}")

Project Root Directory: C:\Users\ping\Files_win10\python\py311\stocks_v0_works
Source file: C:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\df_OHLCV_clean_stocks_etfs.parquet
Pivoting column: 'Adj Close'
Destination file: C:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\df_adj_close.parquet


### Step 1: Load Source OHLCV Data

Load the cleaned, multi-indexed data that will be transformed.

In [2]:
print(f"--- Step 1: Loading source data from {SOURCE_PATH.name} ---")

try:
    df_ohlcv = pd.read_parquet(SOURCE_PATH)
    print("Successfully loaded source data.")
    df_ohlcv.info()
    display(df_ohlcv.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}. Halting execution.")
    df_ohlcv = None
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df_ohlcv = None

--- Step 1: Loading source data from df_OHLCV_clean_stocks_etfs.parquet ---


Successfully loaded source data.
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 547209 entries, ('A', Timestamp('2025-06-26 00:00:00')) to ('ZWS', Timestamp('2024-02-01 00:00:00'))
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Open   547209 non-null  float64
 1   Adj High   547209 non-null  float64
 2   Adj Low    547209 non-null  float64
 3   Adj Close  547209 non-null  float64
 4   Volume     547209 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 23.0+ MB


Adj Open  Adj High  Adj Low  Adj Close   Volume
Ticker Date                                                       
A      2025-06-26    119.18    120.54   118.04     119.42  1486310
       2025-06-25    117.11    119.78   116.97     118.62  1690000
       2025-06-24    117.33    118.00   116.57     117.64  1945300

### Step 2: Extract and Pivot Data

This step performs the core transformation: it selects the specified data column and pivots the DataFrame from a long to a wide format.

In [3]:
def pivot_timeseries_data(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    """
    Selects a column from a multi-indexed DataFrame and pivots it.

    Args:
        df (pd.DataFrame): Input DataFrame with a ('Ticker', 'Date') MultiIndex.
        column_name (str): The name of the column to pivot.

    Returns:
        pd.DataFrame: A wide-format DataFrame with dates as the index and
                      tickers as columns.
    """
    # 1. Select the target column (returns a Series)
    series = df[column_name]
    
    # 2. Unstack the 'Ticker' level (level=0) to become columns
    pivoted_df = series.unstack(level=0)
    
    # 3. Ensure both index (Date) and columns (Ticker) are sorted
    pivoted_df = pivoted_df.sort_index(axis=0).sort_index(axis=1)
    
    return pivoted_df

# --- Execute Step 2 ---
if df_ohlcv is not None:
    print(f"\n--- Step 2: Pivoting '{COLUMN_TO_PIVOT}' column ---")
    df_pivoted = pivot_timeseries_data(df_ohlcv, COLUMN_TO_PIVOT)
    
    print("Transformation complete. New DataFrame info:")
    df_pivoted.info()
    
    print("\nFirst 5 rows of pivoted data:")
    display(df_pivoted.head())
else:
    print("\nSkipping transformation step because source data failed to load.")
    df_pivoted = None


--- Step 2: Pivoting 'Adj Close' column ---

Transformation complete. New DataFrame info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 351 entries, 2024-02-01 to 2025-06-26
Columns: 1559 entries, A to ZWS
dtypes: float64(1559)
memory usage: 4.2 MB

First 5 rows of pivoted data:


Ticker            A       AA    AAL     AAON     AAPL     ABBV     ABEV    ABNB      ABT     ACGL  ACHR      ACI      ACM      ACN     ACWI     ACWV     ACWX    ADBE      ADC      ADI      ADM      ADP    ADSK      ADT      AEE      AEG      AEM      AEP      AER      AES      AFG      AFL   AFRM     AGCO      AGG      AGI     AGNC   AIFU      AIG      AIQ     AIRR      AIT      AIZ      AJG    AKAM       AL      ALB      ALC   ALGM    ALGN    ALK      ALL     ALLE     ALLY    ALNY     ALSN      ALV       AM     AMAT     AMCR     AMD      AME     AMGN      AMH     AMLP      AMP      AMT      AMX    AMZN      AN     ANET    ANSS      AON      AOS      APA      APD    APG      APH       APO    APP    APPF   APTV     AR     ARCC      ARE     ARES    ARGX     ARKB   ARKK    ARM     ARMK     ARW     AS     ASML    ASND      ASR   ASTS      ASX    ATI      ATO      ATR       AU   AUR    AVAV      AVB     AVDE     AVDV     AVEM     AVGO     AVLV   AVTR     AVUS     AVUV      AVY       AWI      AWK    AXON      AXP      AXS   AXTA      AYI   AZEK      AZN      AZO        B      BA     BABA      BAC      BAH     BALL      BAM      BAP      BAX     BBAX     BBCA      BBD     BBEU     BBIN   BBIO     BBJP     BBUS     BBVA     BBWI      BBY      BCE      BCH      BCS      BDX     BEKE      BEN      BEP     BF-A     BF-B    BFAM       BG      BHP    BIDU    BIIB      BIL   BILI     BILS     BINC     BIO      BIP   BIRK   BITB      BIV     BJ       BK     BKLC     BKLN     BKNG      BKR     BLD    BLDR      BLK      BLV      BMI      BMO   BMRN      BMY       BN      BND     BNDX      BNS      BNT   BNTX     BOKF     BOND     BOXX       BP   BPMC     BPOP       BR   BRBR     BRFS     BRK-A   BRK-B     BRKR      BRO   BROS      BRX     BSAC     BSBR     BSCP     BSCQ     BSCR      BSV    BSX      BSY      BTI      BUD   BUFR    BURL      BWA     BWXT       BX      BXP     BXSL      BYD     BZ        C    CACI    CAE      CAG      CAH     CALF     CAR     CARR   CART     CASY      CAT   CAVA       CB    CBOE   CBRE     CBSH   CCCS     CCEP       CCI      CCJ      CCK    CCL   CDE    CDNS      CDP      CDW       CE    CEF      CEG   CELH       CF      CFG   CFLT      CFR       CG     CGCP     CGDV     CGGO     CGGR     CGMU     CGUS     CGXU       CHD     CHDN      CHE    CHKP     CHRD     CHRW    CHT    CHTR   CHWY       CI     CIBR   CIEN     CIGI     CINF       CL     CLH    CLS      CLX       CM      CMA    CMCSA      CME      CMF      CMG      CMI      CMS  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Step 3: Save Pivoted Data

Save the newly created wide-format DataFrame to a Parquet file for future use.

In [4]:
if df_pivoted is not None:
    print(f"\n--- Step 3: Saving pivoted data ---")
    try:
        # Ensure the destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        df_pivoted.to_parquet(DEST_PATH, index=True)
        print(f"Successfully saved pivoted data to: {DEST_PATH}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")
else:
    print("\nSkipping save step because the pivoted DataFrame does not exist.")


--- Step 3: Saving pivoted data ---


Successfully saved pivoted data to: C:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\df_adj_close.parquet


### Step 4: Verify Saved File

To ensure the pipeline ran correctly, load the saved file and display its contents.

In [5]:
print(f"\n--- Step 4: Verifying saved file ---")

try:
    if DEST_PATH.exists():
        verified_df = pd.read_parquet(DEST_PATH)
        print("Verification successful! File read back into memory.")
        print("First 5 rows of verified data:")
        display(verified_df.head())
    else:
        print(f"ERROR: Output file not found at {DEST_PATH}. The save step may have failed.")
except Exception as e:
    print(f"An error occurred during verification: {e}")


--- Step 4: Verifying saved file ---


Verification successful! File read back into memory.
First 5 rows of verified data:


Ticker            A       AA    AAL     AAON     AAPL     ABBV     ABEV    ABNB      ABT     ACGL  ACHR      ACI      ACM      ACN     ACWI     ACWV     ACWX    ADBE      ADC      ADI      ADM      ADP    ADSK      ADT      AEE      AEG      AEM      AEP      AER      AES      AFG      AFL   AFRM     AGCO      AGG      AGI     AGNC   AIFU      AIG      AIQ     AIRR      AIT      AIZ      AJG    AKAM       AL      ALB      ALC   ALGM    ALGN    ALK      ALL     ALLE     ALLY    ALNY     ALSN      ALV       AM     AMAT     AMCR     AMD      AME     AMGN      AMH     AMLP      AMP      AMT      AMX    AMZN      AN     ANET    ANSS      AON      AOS      APA      APD    APG      APH       APO    APP    APPF   APTV     AR     ARCC      ARE     ARES    ARGX     ARKB   ARKK    ARM     ARMK     ARW     AS     ASML    ASND      ASR   ASTS      ASX    ATI      ATO      ATR       AU   AUR    AVAV      AVB     AVDE     AVDV     AVEM     AVGO     AVLV   AVTR     AVUS     AVUV      AVY       AWI      AWK    AXON      AXP      AXS   AXTA      AYI   AZEK      AZN      AZO        B      BA     BABA      BAC      BAH     BALL      BAM      BAP      BAX     BBAX     BBCA      BBD     BBEU     BBIN   BBIO     BBJP     BBUS     BBVA     BBWI      BBY      BCE      BCH      BCS      BDX     BEKE      BEN      BEP     BF-A     BF-B    BFAM       BG      BHP    BIDU    BIIB      BIL   BILI     BILS     BINC     BIO      BIP   BIRK   BITB      BIV     BJ       BK     BKLC     BKLN     BKNG      BKR     BLD    BLDR      BLK      BLV      BMI      BMO   BMRN      BMY       BN      BND     BNDX      BNS      BNT   BNTX     BOKF     BOND     BOXX       BP   BPMC     BPOP       BR   BRBR     BRFS     BRK-A   BRK-B     BRKR      BRO   BROS      BRX     BSAC     BSBR     BSCP     BSCQ     BSCR      BSV    BSX      BSY      BTI      BUD   BUFR    BURL      BWA     BWXT       BX      BXP     BXSL      BYD     BZ        C    CACI    CAE      CAG      CAH     CALF     CAR     CARR   CART     CASY      CAT   CAVA       CB    CBOE   CBRE     CBSH   CCCS     CCEP       CCI      CCJ      CCK    CCL   CDE    CDNS      CDP      CDW       CE    CEF      CEG   CELH       CF      CFG   CFLT      CFR       CG     CGCP     CGDV     CGGO     CGGR     CGMU     CGUS     CGXU       CHD     CHDN      CHE    CHKP     CHRD     CHRW    CHT    CHTR   CHWY       CI     CIBR   CIEN     CIGI     CINF       CL     CLH    CLS      CLX       CM      CMA    CMCSA      CME      CMF      CMG      CMI      CMS  \
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               